# 🔥 Anthropic SDK - Middle Level

## Intermediate Concepts for the Anthropic Python SDK

This notebook builds on the basics and introduces more powerful features.

### What You'll Learn:
- Streaming responses for real-time output
- Temperature and other generation parameters
- Working with images (vision capabilities)
- Counting tokens before sending requests
- Building a conversation manager class

---

## 1. Setup

Import our configuration and create the client.

In [ ]:
import anthropic
from config import MODEL, ANTHROPIC_API_KEY, validate_api_key, DEFAULT_MAX_TOKENS

# Validate and create client
validate_api_key()
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

print(f"Model: {MODEL}")
print("Ready for intermediate concepts!")

## 2. Streaming Responses

Streaming allows you to receive Claude's response in real-time, token by token. This is essential for:
- Better user experience (users see output immediately)
- Long responses that would otherwise timeout
- Interactive applications

In [ ]:
# Basic streaming example
print("Streaming response:")
print("-" * 50)

with client.messages.stream(
    model=MODEL,
    max_tokens=500,
    messages=[{"role": "user", "content": "Write a short poem about coding."}]
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

print("\n" + "-" * 50)
print("Stream complete!")

In [ ]:
# Streaming with event handling for more control
print("Streaming with events:")
print("-" * 50)

collected_text = []

with client.messages.stream(
    model=MODEL,
    max_tokens=300,
    messages=[{"role": "user", "content": "Explain what an API is in 2 sentences."}]
) as stream:
    for text in stream.text_stream:
        collected_text.append(text)
        print(text, end="", flush=True)

# Get the final message with usage stats
final_message = stream.get_final_message()

print("\n" + "-" * 50)
print(f"\nTotal characters streamed: {len(''.join(collected_text))}")
print(f"Input tokens: {final_message.usage.input_tokens}")
print(f"Output tokens: {final_message.usage.output_tokens}")

## 3. Temperature and Generation Parameters

Temperature controls the randomness/creativity of responses:
- `0.0` = Deterministic, focused responses
- `1.0` = Default, balanced creativity
- Higher = More creative/random (max varies by model)

In [ ]:
# Compare different temperatures
prompt = "Give me a creative name for a coffee shop."

print("Temperature Comparison")
print("=" * 60)

for temp in [0.0, 0.5, 1.0]:
    print(f"\nTemperature: {temp}")
    print("-" * 40)
    
    # Run 3 times to show variability
    for i in range(3):
        response = client.messages.create(
            model=MODEL,
            max_tokens=50,
            temperature=temp,
            messages=[{"role": "user", "content": prompt}]
        )
        print(f"  {i+1}. {response.content[0].text.strip()}")

In [ ]:
# Top-p (nucleus sampling) - another way to control randomness
# Lower top_p = more focused, higher = more diverse
response = client.messages.create(
    model=MODEL,
    max_tokens=100,
    temperature=1.0,
    top_p=0.9,  # Only consider tokens in the top 90% probability mass
    messages=[{"role": "user", "content": "Describe a sunset in one sentence."}]
)

print("Using top_p=0.9:")
print(response.content[0].text)

## 4. Stop Sequences

Stop sequences tell Claude when to stop generating. Useful for structured outputs.

In [ ]:
# Using stop sequences
response = client.messages.create(
    model=MODEL,
    max_tokens=200,
    stop_sequences=["END", "---"],  # Stop when Claude outputs these
    messages=[{
        "role": "user", 
        "content": "List 3 programming languages, then write END, then list 3 more."
    }]
)

print("Response (stopped at 'END'):")
print(response.content[0].text)
print(f"\nStop reason: {response.stop_reason}")

## 5. Working with Images (Vision)

Claude can analyze images! You can send images as base64 or URLs.

In [ ]:
import base64
import httpx

# Method 1: Image from URL
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png"

# Fetch and encode the image
image_data = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")

response = client.messages.create(
    model=MODEL,
    max_tokens=300,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": image_data,
                    },
                },
                {
                    "type": "text",
                    "text": "What is this logo? Describe it briefly."
                }
            ],
        }
    ],
)

print("Image Analysis:")
print("-" * 50)
print(response.content[0].text)

In [ ]:
# Helper function for image analysis
def analyze_image_from_url(image_url, question, media_type="image/png"):
    """Analyze an image from a URL."""
    image_data = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")
    
    response = client.messages.create(
        model=MODEL,
        max_tokens=500,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": media_type,
                            "data": image_data,
                        },
                    },
                    {"type": "text", "text": question}
                ],
            }
        ],
    )
    return response.content[0].text

# Example usage
result = analyze_image_from_url(
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Typescript_logo_2020.svg/200px-Typescript_logo_2020.svg.png",
    "What programming language does this logo represent?"
)
print(result)

## 6. Token Counting

Count tokens before sending requests to estimate costs and stay within limits.

In [ ]:
# Count tokens using the API
messages = [
    {"role": "user", "content": "What is the meaning of life, the universe, and everything?"}
]

# Count tokens before sending
token_count = client.messages.count_tokens(
    model=MODEL,
    messages=messages
)

print(f"Input tokens: {token_count.input_tokens}")

# Now make the actual request
response = client.messages.create(
    model=MODEL,
    max_tokens=100,
    messages=messages
)

print(f"Actual input tokens used: {response.usage.input_tokens}")
print(f"Output tokens used: {response.usage.output_tokens}")
print(f"\nResponse: {response.content[0].text}")

## 7. Building a Conversation Manager

Let's create a reusable class to manage conversations with Claude.

In [ ]:
class ConversationManager:
    """Manage multi-turn conversations with Claude."""
    
    def __init__(self, client, model=MODEL, system=None, max_tokens=DEFAULT_MAX_TOKENS):
        self.client = client
        self.model = model
        self.system = system
        self.max_tokens = max_tokens
        self.messages = []
        self.total_input_tokens = 0
        self.total_output_tokens = 0
    
    def chat(self, user_message, stream=False):
        """Send a message and get a response."""
        # Add user message
        self.messages.append({"role": "user", "content": user_message})
        
        # Build request kwargs
        kwargs = {
            "model": self.model,
            "max_tokens": self.max_tokens,
            "messages": self.messages
        }
        if self.system:
            kwargs["system"] = self.system
        
        if stream:
            return self._stream_response(kwargs)
        else:
            return self._get_response(kwargs)
    
    def _get_response(self, kwargs):
        """Get a non-streaming response."""
        response = self.client.messages.create(**kwargs)
        
        # Track tokens
        self.total_input_tokens += response.usage.input_tokens
        self.total_output_tokens += response.usage.output_tokens
        
        # Add assistant response to history
        assistant_message = response.content[0].text
        self.messages.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def _stream_response(self, kwargs):
        """Get a streaming response."""
        collected_text = []
        
        with self.client.messages.stream(**kwargs) as stream:
            for text in stream.text_stream:
                collected_text.append(text)
                print(text, end="", flush=True)
            
            final_message = stream.get_final_message()
            self.total_input_tokens += final_message.usage.input_tokens
            self.total_output_tokens += final_message.usage.output_tokens
        
        assistant_message = "".join(collected_text)
        self.messages.append({"role": "assistant", "content": assistant_message})
        print()  # New line after streaming
        
        return assistant_message
    
    def get_stats(self):
        """Get conversation statistics."""
        return {
            "message_count": len(self.messages),
            "total_input_tokens": self.total_input_tokens,
            "total_output_tokens": self.total_output_tokens,
            "total_tokens": self.total_input_tokens + self.total_output_tokens
        }
    
    def clear(self):
        """Clear conversation history."""
        self.messages = []
        self.total_input_tokens = 0
        self.total_output_tokens = 0

print("ConversationManager class defined!")

In [ ]:
# Use the ConversationManager
conv = ConversationManager(
    client, 
    system="You are a helpful Python tutor. Keep responses concise."
)

# Have a conversation
print("Message 1:")
print(conv.chat("What is a list in Python?"))
print("\n" + "=" * 50 + "\n")

print("Message 2:")
print(conv.chat("How do I add items to it?"))
print("\n" + "=" * 50 + "\n")

print("Message 3:")
print(conv.chat("What did we discuss first?"))
print("\n" + "=" * 50 + "\n")

# Check stats
print("Conversation Statistics:")
print(conv.get_stats())

## 8. Practice Exercises

In [ ]:
# Exercise 1: Create a streaming response that writes a haiku
# Your code here:



In [ ]:
# Exercise 2: Compare temperature=0 vs temperature=1 for generating a product tagline
# Your code here:



In [ ]:
# Exercise 3: Extend ConversationManager to support saving/loading conversations
# Your code here:



## Summary

In this notebook, you learned:

1. **Streaming** - Real-time token-by-token responses
2. **Temperature** - Controlling creativity and randomness
3. **Top-p sampling** - Alternative randomness control
4. **Stop sequences** - Custom stopping conditions
5. **Vision** - Analyzing images with Claude
6. **Token counting** - Estimating costs before requests
7. **Conversation management** - Building reusable conversation classes

---

**Next:** Move on to `03_advanced_level.ipynb` to learn about tool use, JSON mode, and advanced patterns!